In [10]:
# imports and functions, does nothing

from functools import partial
from IPython.display import Image, display, Video, Image, HTML
from matplotlib import rcParams
from prettytable import PrettyTable
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils.multiclass import unique_labels
import glob
import json
import sys
sys.path.append('../..')
import lib.plot_builder as plot_builder
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas
import pathlib
import random
import sklearn
import random
import matplotlib
matplotlib.rcParams.update({'font.size': 14})

def print_table(data, fields=None):
    t = PrettyTable()
    for row in data:
        t.add_row(row)
    t.align = 'l'
    if fields != None:
        t.field_names = fields
    print(t)
    
def read_dataset(f):
    acc = None
    precision = None
    recall = None
    f1score = None
    with open(f) as f2:
        data = json.load(f2)
        acc = round(100*data['score']['accuracy'][0], 1)
        precision = round(100*data['score']['precision'][0], 1)
        recall = round(100*data['score']['recall'][0], 1)
        f1score = round(100*data['score']['f1score'][0], 1)

    cm = f.replace('datasets/', 'plots/').replace('.json', '-cm.png')
    fi = f.replace('datasets/', 'plots/').replace('.json', '-fi.png')

    return acc, precision, recall, f1score, cm, fi
def img(path, width=400):
    rnd = random.randint(0,2e9)
    return f"""<img src="{path}?nocache={rnd}" style="width:{width}px; "></img>"""

def latex_table(table, header):
    header_bold = ["\\textbf{"+t+"}" for t in header]
    table2 = [header_bold]
    table2.extend(table)
    latex_table = """\\begin{tabular}{lrrrr}
"""
    rows = [" & ".join(map(str,row)) for row in table2]
    latex_table += "".join(["    " + row + " \\\\\n" for row in rows])
    latex_table += """\\end{tabular}"""

    return latex_table

def dataset_name_to_friendly_name(name):
    root_dataset = name.replace('-netflow1000', '').replace('-netflow100', '').replace('-netflow10', '').replace('-netflows', '').replace('-netflow', '').replace('-defended', '').replace('-google', '')
    nicename = name.replace('-netflow1000', "NF 0.1%").replace('-netflow100', "NF   1%").replace('-netflow10', "NF  10%").replace('-netflow', "NF 100%").replace('/', ' ').replace('-defended', ' (defended)').replace('-google', '(Google dest filter)')
    variant = nicename.replace(root_dataset, '')
    return root_dataset, nicename, variant

def find_pos(name, features):
    i = 0
    while i<len(features):
        if features[i].strip() == name:
            return i
        i += 1
    print("Couldn't find", name)
    sys.exit(0)

In [13]:
#!/usr/bin/python3

import os
import sys
import glob
import src.features_from_pcaps as kffeatures_pcaps


def build_if_not_exists(path, output):
    output = 'datasets/' + output
    if not os.path.isfile(output):
        print(f"Working on {output}")
        kffeatures_pcaps.build(path+"/npys/summary_quic_tls_merged.npy", output_name=output, defense=None)
    else:
        print(f"{output} already exists, skipping")


build_if_not_exists('.pcaps/quic-100p-150-google', 'quic-100p-150.npy')
print("Done")

Working on datasets/quic-100p-150.npy
[[0, 0.0, 1], [0, 5.787376, 1], [0, 5.864717, 1], [0, 5.870109, 1], [0, 5.872131, 1], [0, 5.874166, 1], [0, 5.895504, 1]]
[[0, 0.0, 1], [0, 0.786672, 1], [0, 0.789175, 1], [0, 0.792508, 1], [0, 0.796914, 1], [0, 0.800652, 1], [0, 0.817521, 1]]
[[0, 0.0, 1], [0, 0.842351, 1], [0, 0.845105, 1], [0, 0.852763, 1], [0, 0.859037, 1], [0, 0.871522, 1], [0, 0.873782, 1]]
[[0, 0.0, 1], [0, 4.365895, 1], [0, 4.368418, 1], [0, 4.370797, 1], [0, 4.372828, 1], [0, 4.374844, 1], [0, 4.384665, 1]]
[[0, 0.0, 1], [0, 5.484818, 1], [0, 5.487291, 1], [0, 5.489352, 1], [0, 5.497487, 1], [0, 5.513767, 1], [0, 5.515981, 1]]
[[0, 0.0, 1], [0, 17.456661, 1], [0, 17.458965, 1], [0, 17.461056, 1], [0, 17.463992, 1], [0, 17.476571, 1], [0, 17.498655, 1]]
[[0, 0.0, 1], [0, 0.844711, 1], [0, 0.84789, 1], [0, 0.850461, 1], [0, 0.853627, 1], [0, 0.86037, 1], [0, 0.86488, 1]]
[[0, 0.0, 1], [0, 6.303093, 1], [0, 6.305524, 1], [0, 6.312658, 1], [0, 6.31477, 1], [0, 6.328223, 1], [0

In [14]:
#!/usr/bin/python3

import os
import sys
import glob
import src.attack as attack
import lib.plot_builder as plot_builder

npys = glob.glob('datasets/*.npy', recursive=False)

for npy_file in npys:
    json_file = npy_file.replace('.npy', '.json')

    if os.path.isfile(json_file):
        print("Skipping", npy_file)
        continue
    
    results = attack.run(npy_file)
    plot_builder.serialize(json_file, results)

print("Done")


Skipping datasets/quic-100p-338.npy
Running attack on datasets/quic-100p-150.npy
Writting map plots/quic-100p-150.map.json
Loaded 4920 rows, 244 feature names for 244 features
Number of classes 123
Number of features 244
Number of samples 4920
Number of labels 4920
125 but only found 123
Scores were 0.6653963414634146 but were corrected to 0.65475.
125 but only found 123
Scores were 0.6676829268292683 but were corrected to 0.657.
125 but only found 123
Scores were 0.6679369918699187 but were corrected to 0.65725.
125 but only found 123
Scores were 0.6717479674796748 but were corrected to 0.661.
125 but only found 123
Scores were 0.6770833333333334 but were corrected to 0.66625.
125 but only found 123
Scores were 0.6755589430894309 but were corrected to 0.66475.
125 but only found 123
Scores were 0.6661585365853658 but were corrected to 0.6555.
125 but only found 123
Scores were 0.6786077235772358 but were corrected to 0.66775.
125 but only found 123
Scores were 0.6692073170731707 but w

In [16]:
# Size of record
import numpy as np
import sys

data = np.load('.pcaps/quic-100p-150-google/npys/summary_quic_tls_merged.npy', allow_pickle=True).item()

sizes = dict()

for url in data:
    for sample in data[url]['quic']:
        pkts = [x for x in data[url]['quic'][sample] if x[2]>0]
        if len(pkts) == 0:
            continue

        if not url in sizes:
            sizes[url] = []

        number_of_hellos = len(pkts)
        sizes[url].append(number_of_hellos)

tuples = []
for url in sizes:
    tuples.append((url, np.mean(sizes[url])))

tuples.sort(key=lambda x: x[1])

values = [x[1] for x in tuples]
print(f"Number of client hellos: min {np.min(values)}, mean {np.mean(values)}, max {np.max(values)}")

Number of client hellos: min 1.0, mean 8.413393103448275, max 27.0


In [6]:
acc, prec, recall, f1, cm, fi =  read_dataset('datasets/quic-100p-150.json')

print(f1)

display(HTML(img(cm)))
display(HTML(img(fi)))

43.0


In [33]:
# loads datasets of interest

import sys
import os
import glob
import numpy as np
from pathlib import Path

def getsizesnetflows(name, dataset_path):
    return getsizespcap(name, dataset_path, filterstr="**/nfcapd*")

def getsizespcap(dataset_path, filterstr="**/*.pcap"):
    pcaps = glob.glob(dataset_path + filterstr, recursive=True)

    sizes = dict()
    for p in pcaps:
        f = p.replace(dataset_path, '')
        parts = f.split('/')
        url = parts[0]

        if not url in sizes:
            sizes[url] = []

        size = os.path.getsize(p)
        sizes[url].append(size)

    keys = list(sizes.keys())
    means = []
    
    for k in keys:
        mean = round(np.mean(sizes[k])/1024, 2)
        means.append(mean)

    return round(np.mean(means),2)

tuples = []

acc, precision, recall, f1score, cm, fi = read_dataset('../randomforests/datasets/quic-100p-150.json')
size = getsizespcap('../../cf-clusters-datasets/quic-100p-150/pcaps/')
tuples.append(["Baseline (Full feature set on all traffic)", f1score, size])

acc, precision, recall, f1score, cm, fi = read_dataset('../randomforests-googleview/datasets/quic-100p-150-google.json')
size = getsizespcap('../../cf-clusters-datasets/quic-100p-150-google/pcaps/')
tuples.append(["Full feature set on Google traffic", f1score, size])

acc, precision, recall, f1score, cm, fi = read_dataset('datasets/quic-100p-150.json')
size = round(7.6 * 64 / 8 / 1024,1)
tuples.append(["Timings of \\texttt{ClientHello} to Google", f1score, size])

table_data=tuples

print(f"table_data = ", tuples)

print_table(table_data, fields=["Variant", "F1 Score", "Storage"])
print(latex_table(table_data, header=["Variant", "F1 Score", "Storage"]))

table_data =  [['Baseline (Full feature set on all traffic)', 95.8, 312.42], ['Full feature set on Google traffic', 78.4, 112.15], ['Timings of \\texttt{ClientHello} to Google', 77.9, 0.1]]
+--------------------------------------------+----------+---------+
| Variant                                    | F1 Score | Storage |
+--------------------------------------------+----------+---------+
| Baseline (Full feature set on all traffic) | 95.8     | 312.42  |
| Full feature set on Google traffic         | 78.4     | 112.15  |
| Timings of \texttt{ClientHello} to Google  | 77.9     | 0.1     |
+--------------------------------------------+----------+---------+
\begin{tabular}{lrrrr}
    \textbf{Variant} & \textbf{F1 Score} & \textbf{Storage} \\
    Baseline (Full feature set on all traffic) & 95.8 & 312.42 \\
    Full feature set on Google traffic & 78.4 & 112.15 \\
    Timings of \texttt{ClientHello} to Google & 77.9 & 0.1 \\
\end{tabular}
